In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "ab040-other-sdks"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='06d6c79f-e74e-42af-a8ca-36d5ed991822', created_at=datetime.datetime(2025, 7, 28, 3, 58, 3, 842049, tzinfo=TzInfo(UTC)), name='7a80a27a-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 28, 3, 58, 3, 842049, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='8b5edcdb-ca36-423d-88e5-0f3f7b42f2e5', agent_id='cd4b9256-144b-4aef-949b-f9000995013b', sequence_id=245, task_id='06d6c79f-e74e-42af-a8ca-36d5ed991822', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 28, 3, 58, 4, 299604, tzinfo=TzInfo(UTC)))


In [6]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=20,
)

╭───────────────────────── USER [07/28/2025 03:58:04] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 03:58:16] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest news about AI and AI startups",                          │
│    "model": "gpt-4o-mini",                                                   │
│    "type": "web_search_preview"                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 03:58:16] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "Artificial intelligence (AI) continues to be a driving force in  │
│  the startup ecosystem, with significant investments and strategic           │
│  partnerships shaping the landscape.\n\n**Surge in AI Startup Funding**\n\n  │
│  the first half of 2025, U.S. AI startups experienced a substantial funding  │
│  surge, with investments reaching $162.8 billion\u2014a 75.6% increase from  │
│  the previous year. This growth was propelled by major tech firms'           │
│  investments, including OpenAI's $40 billion funding round and Meta's $14.3  │
│  billion investment in Scale AI. Notably, AI-related investments accounted   │
│  for 64.1% of total deal value during this period.                           │
│  ([reuters.com](https://www.reuters.com/business/us-ai-startups-see-funding  │
│  urge-while-more-vc-funds-struggle-raise-data-shows-2025-07-15/?utm_source=  │
│  enai))\n\n**Dominance of AI in Venture Capital**\n\nAI startups have becom  │
│  dominant in global venture capital funding, securing 53% of all investment  │
│  worldwide and 64% in the U.S. in the first half of 2025. This trend reflec  │
│  a transformative shift in tech investment, with AI's potential driving      │
│  significant capital allocation.                                             │
│  ([axios.com](https://www.axios.com/2025/07/03/ai-startups-vc-investments?u  │
│  _source=openai))\n\n**Strategic Partnerships and Investments**\n\n- **Amaz  │
│  and Anthropic**: Amazon increased its investment in AI startup Anthropic b  │
│  an additional $4 billion, bringing its total investment to $8 billion.      │
│  Despite this substantial backing, Amazon remains a minority investor. The   │
│  partnership aims to foster innovation and responsible AI development, with  │
│  Anthropic collaborating closely with Amazon Web Services (AWS) and utilizi  │
│  AWS chips for AI model advancement.                                         │
│  ([apnews.com](https://apnews.com/article/7a5764907e8cf0c23117be9c710e9f6a?  │
│  m_source=openai))\n\n- **Applied Intuition**: In June 2025, Applied         │
│  Intuition achieved a $15 billion valuation after completing a $600 million  │
│  Series F funding round and tender offer. Investors include Andreessen       │
│  Horowitz, Fidelity Investments, Kleiner Perkins, Microsoft's venture capit  │
│  arm M12, and Porsche Investments Management.                                │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Applied_Intuition?utm_so  │
│  ce=openai))\n\n- **Mistral AI**: The French AI startup Mistral AI secured   │
│  \u20ac600 million ($645 million) funding round in June 2024, elevating its  │
│  valuation to \u20ac5.8 billion ($6.2 billion). Led by venture capital firm  │
│  General Catalyst, this round aims to support the company's expansion.       │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Mistral_AI?utm_source=op  │
│  ai))\n\n**Emerging AI Startups**\n\n- **Neysa**: Founded in 2023 by Sharad  │
│  Sanghi and Anindya Das, Neysa is an Indian technology startup providing a   │
│  cloud platform for AI acceleration and high-performance computing           │
│  infrastructure services. The company raised $20 million in a seed funding   │
│  round in February 2024, followed by a $30 million round in October 2024,    │
│  bringing its total funding to

╭──────────────────────── AGENT [07/28/2025 03:58:17] ─────────────────────────╮
│ Here are some of the latest developments in the world of AI and AI startups: │
│                                                                              │
│  1 Surge in AI Startup Funding:                                              │
│     • In the first half of 2025, U.S. AI startups saw a funding surge with   │
│       investments hitting $162.8 billion. This growth was driven by major    │
│       investments from tech firms such as OpenAI and Meta. AI-related        │
│       investments made up 64.1% of the total deal value in this period.      │
│  2 Dominance in Venture Capital:                                             │
│     • AI startups have secured 53% of global venture capital funding, with   │
│       64% in the U.S., underscoring AI's increasing significance in tech     │
│       investment.                                                            │
│  3 Strategic Partnerships and Investments:                                   │
│     • Amazon and Anthropic: Amazon increased its investment in Anthropic to  │
│       $8 billion. This partnership focuses on innovation and responsible AI  │
│       development.                                                           │
│     • Applied Intuition: Achieved a $15 billion valuation after a $600       │
│       million funding round.                                                 │
│     • Mistral AI: Raised €600 million, aiming to support expansion.          │
│  4 Emerging AI Startups:                                                     │
│     • Neysa: An Indian startup providing cloud platforms for AI, raised $50  │
│       million.                                                               │
│     • Axelera AI: Netherlands-based, received a €61.6 million grant for its  │
│       AI processing units.                                                   │
│  5 AI's Impact on Various Sectors:                                           │
│     • AI is being used to expedite processes in industries like nuclear      │
│       energy, where partnerships aim to accelerate reactor licensing and     │
│       construction.                                                          │
│                                                                              │
│ For more detailed articles and specific company information, you can check   │
│ the sources like Reuters, Axios, and AP News.                                │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 20 seconds - returning collected messages
